# Naive representation pipeline (qiskit)

Main steps:

* input: image of the sky (frequency domain)
* apply 2D Fourier transform --> visibilities (Fourier domain)
* encode classical visibilities data into qbits (non-conventional domain)
* measure qbits (back to conventional domain)
* apply 2D Inverse Fourier transform --> original image?

In [22]:
import numpy as np
import sys
import struct

import qiskit
from qiskit import *
from qiskit.visualization import plot_histogram

## Generating an artificial image of the sky

In [30]:
#image of the sky filled with single precision complex floats
#pixels are set to low complex random values (image background) 
#few pixels are set to larger complex random values in a specified ellipse area (image source/subject)

n = 4
image = np.zeros((n, n), dtype='complex64')
image.real = np.random.rand(n , n) / 100
image.imag = np.random.rand(n , n) / 100

h, w = image.shape
mask = circular_mask(h, w, radius=h/2)
sky_image = image.copy()
sky_image[~mask] = complex(np.random.rand() * 100, np.random.rand() * 100)
print(sky_image)

[[1.1381033e+01+7.4026764e+01j 1.1381033e+01+7.4026764e+01j
  5.5577266e-03+6.7168823e-04j 1.1381033e+01+7.4026764e+01j]
 [1.1381033e+01+7.4026764e+01j 9.0924306e-03+2.2347141e-03j
  8.4902812e-03+7.9622259e-03j 3.3545624e-03+1.1026422e-03j]
 [6.5371674e-03+8.7434370e-03j 1.8683868e-03+7.2393091e-03j
  3.6867277e-03+1.2568048e-03j 6.8683354e-03+3.4162225e-03j]
 [1.1381033e+01+7.4026764e+01j 7.8385286e-03+2.8767535e-03j
  5.7907579e-03+5.6034480e-03j 2.4548810e-04+1.8364440e-04j]]


## Applying 2D Fourier transform (visibilities)

In [33]:
visibilities = np.fft.fft2(sky_image).astype('complex64')
print(visibilities)

[[ 56.964497+370.1751j    34.13376 +222.06522j   11.381828 +74.03395j
   34.11846 +222.08188j ]
 [ 34.13233 +222.05324j   11.368299 +74.01787j  -11.374881 -74.02377j
   11.372234 +74.02474j ]
 [ 11.370739 +74.02813j  -11.369461 -73.98805j  -34.129803-222.08745j
  -11.369457 -74.02471j ]
 [ 34.12706 +222.06737j   11.369306 +74.009346j -11.379045 -74.0271j
   11.380662 +74.02247j ]]


### Sanity check

In [43]:
img = np.fft.ifft2(visibilities).astype('complex64')

print(img)

test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()
 
print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[1.1381033e+01+7.4026764e+01j 1.1381033e+01+7.4026764e+01j
  5.5578351e-03+6.6995621e-04j 1.1381033e+01+7.4026764e+01j]
 [1.1381033e+01+7.4026764e+01j 9.0927482e-03+2.2338629e-03j
  8.4905624e-03+7.9619288e-03j 3.3550858e-03+1.1023283e-03j]
 [6.5372586e-03+8.7447166e-03j 1.8687248e-03+7.2402358e-03j
  3.6870837e-03+1.2564659e-03j 6.8678856e-03+3.4170747e-03j]
 [1.1381033e+01+7.4026764e+01j 7.8383088e-03+2.8764009e-03j
  5.7907104e-03+5.6031346e-03j 2.4551153e-04+1.8274784e-04j]]

Reals MSE: 6.0531455e-14
Imaginaries MSE: 5.17173e-13


## Classical data encoding/decoding

In [55]:
number_of_entries = visibilities.shape[0] * visibilities.shape[1]
number_of_bits_real = 32 #since single precision complex floats
number_of_bits_imag = 32
number_of_bits = number_of_bits_real + number_of_bits_imag
off_set = 0

qc = QuantumCircuit(number_of_entries*number_of_bits) #n bits encoded into n qbits (circuit family #1)
for i in range(0, visibilities.shape[0]):
    for j in range(0, visibilities.shape[1]):
            binary_real = float_to_bin_real(visibilities[i, j].real)
            binary_imag = float_to_bin_imag(visibilities[i, j].imag)
            binary = binary_real+binary_imag
            off_set = encoding2(qc, binary, off_set)

#measurement
qc.measure_all()
backend = Aer.get_backend('aer_simulator')
job = backend.run(qc, shots=1, memory=True)
output = job.result().get_memory()[0]
out = reverse(output)

#readout
chunks_real = []
chunks_imag = []
for i in range(0, number_of_entries):
    chunks_real.append(out[number_of_bits*i:(number_of_bits*i)+number_of_bits_real]) #real parts represented every 64 bits starting from the first one
    chunks_imag.append(out[(number_of_bits*i)+number_of_bits_imag:(number_of_bits*i)+number_of_bits_imag+number_of_bits_imag]) #imaginary parts represented every 64 bits starting after the first occurence of a real part 

readout = []
for i in range(0, len(chunks_real)):
    readout.append(complex(bin_to_float_real(chunks_real[i]), bin_to_float_imag(chunks_imag[i]))) 

#reshaping the readout vector into a nxn matrix
readout = np.array(readout).reshape(n , n).astype('complex64')
print(readout)

[[ 56.964497+370.1751j    34.13376 +222.06522j   11.381828 +74.03395j
   34.11846 +222.08188j ]
 [ 34.13233 +222.05324j   11.368299 +74.01787j  -11.374881 -74.02377j
   11.372234 +74.02474j ]
 [ 11.370739 +74.02813j  -11.369461 -73.98805j  -34.129803-222.08745j
  -11.369457 -74.02471j ]
 [ 34.12706 +222.06737j   11.369306 +74.009346j -11.379045 -74.0271j
   11.380662 +74.02247j ]]


## Applying 2D Inverse Fourier transform (+ fidelity test)

In [60]:
img = np.fft.ifft2(readout).astype('complex64')
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()

print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[1.1381033e+01+7.4026764e+01j 1.1381033e+01+7.4026764e+01j
  5.5578351e-03+6.6995621e-04j 1.1381033e+01+7.4026764e+01j]
 [1.1381033e+01+7.4026764e+01j 9.0927482e-03+2.2338629e-03j
  8.4905624e-03+7.9619288e-03j 3.3550858e-03+1.1023283e-03j]
 [6.5372586e-03+8.7447166e-03j 1.8687248e-03+7.2402358e-03j
  3.6870837e-03+1.2564659e-03j 6.8678856e-03+3.4170747e-03j]
 [1.1381033e+01+7.4026764e+01j 7.8383088e-03+2.8764009e-03j
  5.7907104e-03+5.6031346e-03j 2.4551153e-04+1.8274784e-04j]]

Reals MSE: 6.0531455e-14
Imaginaries MSE: 5.17173e-13


## Utils

### Quantum

In [7]:
def encoding2(qc, binary, off_set):
    
    for i in range(0, len(binary)):
        qc.reset(off_set+i)

        if binary[i]=='1':
            qc.x(off_set+i)
    
    off_set += len(binary)

    return off_set

### Classical

In [2]:
#float to binary / binary to float
def float_to_bin_real(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')
def float_to_bin_imag(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')

def bin_to_float_real(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]
def bin_to_float_imag(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]

#reverse a string (for the measurement step readout)
def reverse(string):
    string = string[::-1]
    return string

#creates a circular mask over a 2D array
def circular_mask(h, w, center=None, radius=None):
    if center is None: #image center
        center = (int(w/2), int(h/2))
    if radius is None: #smallest distance between center and image bounderies
        radius = min(center[0], center[1], w-center[0], h-center[1])
        
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    
    return mask